In [1]:
import pandas as pd
import requests
import numpy as np
import os
import geopandas as gpd
from shapely.geometry import Point
import csv

In [52]:
def df_catolicos(): 
    templos_catolicos = requests.get('https://datos.madrid.es/egob/catalogo/209426-0-templos-catolicas.json')#llamada a la api
    templos_catolicos = templos_catolicos.json()#leerlo como json
    df1 = pd.json_normalize(templos_catolicos['@graph'])#escoger solo el diccionario graph dentro del json
    df1 = df1.drop(['@id', '@type', 'id', 'relation', 'address.district.@id', 'address.area.@id', 'address.postal-code', 'organization.organization-desc', 'organization.accesibility', 'organization.schedule', 'organization.services', 'organization.organization-name'], axis=1)#lmipieza
    df1 = df1.rename(columns={'title':'Place of interest', 'address.locality':'City', 'address.street-address':'Place address', 'location.latitude':'lat_start', 'location.longitude':'long_start'})
    df1["Type of place"] = "catolicas"
    return df1

In [55]:
def df_no_catolicos():
    templos_no_catolicos = requests.get('https://datos.madrid.es/egob/catalogo/209434-0-templos-otros.json')
    templos_no_catolicos = templos_no_catolicos.json()
    df2 = pd.json_normalize(templos_no_catolicos['@graph'])
    df2 = df2.drop(['@id', '@type', 'id', 'relation', 'address.district.@id', 'address.area.@id', 'address.postal-code', 'organization.organization-desc', 'organization.accesibility', 'organization.schedule', 'organization.services', 'organization.organization-name'], axis=1)#limpieza
    df2 = df2.rename(columns={'title':'Place of interest', 'address.locality':'City', 'address.street-address':'Place address', 'location.latitude':'lat_start', 'location.longitude':'long_start'})
    df2["Type of place"] = "no_catolicas"
    return df2

In [56]:
def df_bicimad():
    df3 = pd.read_json("../data/bicimad_station.json")
    df3 ['LONGITUD'] = [float(index.split(',')[0].replace("[", "")) for index in df3['geometry_coordinates']]
    df3 ['LATITUD'] = [float(index.split(',')[1].replace("]", "")) for index in df3['geometry_coordinates']]
    df_bicimad = df3.drop(['id', 'light', 'number', 'activate', 'no_available', 'total_bases', 'dock_bikes', 'free_bases', 'reservations_count', 'geometry_type'], axis=1)#limpieza
    df_bicimad = df3.rename(columns={'name': 'BiciMAD station', 'address': 'Station location', 'LONGITUD':'long_finish', 'LATITUD':'lat_finish'})#rename
    return df_bicimad

In [57]:
df_no_catolicos()

,Place of interest,City,Place address,lat_start,long_start,Type of place
0,Asambleas de Evangelización Mundial para Crist...,MADRID,CALLE ANTONIA RODRIGUEZ SACRISTAN 8,40.372800,-3.745074,no_catolicas
1,Asociación Darse - Dar Ayuda para la Reinserci...,MADRID,CAMINO ALTO DE SAN ISIDRO 16 PLANTA 2º C C,40.396470,-3.722813,no_catolicas
2,Asociación Tiempo Para Cuidar,MADRID,CALLE GENERAL LACY 24 PLANTA BAJA A,40.403234,-3.690948,no_catolicas
3,Asociación Visión Internacional,MADRID,CALLE DISCOBOLO 53,40.442728,-3.612983,no_catolicas
4,Casa sobre la Roca Iglesia Cristiana Integral ...,MADRID,CALLE ULISES 1,40.456372,-3.655613,no_catolicas
...,...,...,...,...,...,...
142,Testigos Cristianos de Jehová. Calle Puerto de...,MADRID,CALLE PUERTO DE POZAZAL 1 PLANTA BAJA,40.379417,-3.629058,no_catolicas
143,Testigos Cristianos de Jehová. Calle San Moisés,MADRID,CALLE SAN MOISÉS 39 PLANTA BAJA B,40.382078,-3.665580,no_catolicas
144,Testigos Cristianos de Jehová. Calle Tembleque,MADRID,CALLE TEMBLEQUE 64 PLANTA BAJA,40.395189,-3.760301,no_catolicas
145,Testigos Cristianos de Jehová. Calle Venancio ...,MADRID,CALLE VENANCIO MARTIN 19 local 2,40.397957,-3.661471,no_catolicas


In [58]:
def concat():
    frames = [df_catolicos(), df_no_catolicos()]
    df_origen = pd.concat(frames)
    return df_origen

In [60]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(float(lat), float(long))], crs=4326)
    c = c.to_crs(3857)
    return c

In [61]:
def merge():
    concat() ['Start'] = concat().apply(lambda x: to_mercator(x['lat_start'], x['long_start']), axis =1)
    df_bicimad() ['final'] = df_bicimad().apply(lambda x: to_mercator(x['lat_finish'], x['long_finish']), axis =1)
    df_merge = pd.merge(concat(), df_bicimad(), how = 'cross')
    return(df_merge)

In [66]:
merge().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140976 entries, 0 to 140975
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Place of interest     140976 non-null  object 
 1   City                  140976 non-null  object 
 2   Place address         140976 non-null  object 
 3   lat_start             140712 non-null  float64
 4   long_start            140712 non-null  float64
 5   Type of place         140976 non-null  object 
 6   id                    140976 non-null  int64  
 7   BiciMAD station       140976 non-null  object 
 8   light                 140976 non-null  int64  
 9   number                140976 non-null  object 
 10  Station location      140976 non-null  object 
 11  activate              140976 non-null  int64  
 12  no_available          140976 non-null  int64  
 13  total_bases           140976 non-null  int64  
 14  dock_bikes            140976 non-null  int64  
 15  

In [63]:
def distance_meters(Start, final):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    return Start.distance(final)

In [64]:
def tabla_maestra():
    tabla_maestra = merge()["Distance"] = merge().apply(lambda x: distance_meters(x['Start'], x['final']), axis =1)
    return tabla_maestra

In [65]:
tabla_maestra()

KeyError: 'Start'